# Interactive maps¶
In this tutorial we will learn how to publish data from Python on interactive leaflet.js maps.

JavaScript (JS) is a programming language for adding interactive content (such a zoomamble maps!) on webpages. Leaflet is a popular JavaScript library for creating interactive maps for webpages (OpenLayers is another JavaScript library for the same purpose).

Here, will mainly focus on Folium - a Python library that makes it easy to convert data from (Geo)DataFrames into interactive Leaflet maps.

Other interesting libraries for creating interactive visualizations from spatial data:

- mapboxgl

- Bokeh

- Geoviews

- plotly express

# Creating a simple interactive web-map¶
- Import folium and other useful packages:

In [1]:
import folium

In [2]:
from pyproj import CRS
import geopandas as gpd
import matplotlib.pyplot as plt

We will start by creating a simple interactive web-map without any data on it. We just visualize OpenStreetMap on a specific location of the world.

- First thing that we need to do is to create a Map instance and define a location for zooming in the data:

In [4]:
#create a map instance
m = folium.Map(location= [ 60.25, 24.8 ] , zoom_start=10, control_scale=True)

- The first parameter location takes a pair of lat, lon values as list as an input which will determine where the map will be positioned when user opens up the map. 
- zoom_start -parameter adjusts the default zoom-level for the map (the higher the number the closer the zoom is). 
- control_scale defines if map should have a scalebar or not.

Let’s see what our map looks like:

In [5]:
m

In [6]:
outfp = "base_map.html"
m.save(outfp)

- You should now see a html file in your working directory. 
- You can open the file in a web browser in order to see the map or in a text editor in order to see the source definition

Let’s create another map with different settings (location, bacground map, zoom levels etc). 
See documentation of the Map() object for all avaiable options.

- tiles -parameter is used for changing the background map provider and map style (see the documentation for all in-built options).

In [8]:
#change the base map to 'Stamen Toner'
m = folium.Map(location=[40.730610, -73.935242], tiles = 'Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True )

m

# Adding layers to the map

- Add a simple marker on the webmap

In [9]:
#Create a map instance
m= folium.Map(location= [60.20, 24.96],
                  zoom_start = 12, 
                      control_scale=True)

#add marker 
#run help(folium.Icon) for more info about icons

#create a marker at 'location lat long, 
#text_box = Kumpula Campus, 
#symobology= green marker with ok_sign inside
#add to the map
folium.Marker(
                location= [60.20426,24.96179],
                    popup="Kumpula Campus",
                        icon=folium.Icon(color='green', icon='ok-sign') ).add_to(m)

#show on map
m

 Folium combines the strenghts of data manipulation in Python with the mapping capabilities of Leaflet.js.

- plot data from a geodataframe using folium.

In [12]:
#file path
points_fp = r"addresses.shp"

#read the data
points= gpd.read_file(points_fp)

#check the input data
points.head()

,address,id,geometry
0,"Kampinkuja 1, 00100 Helsinki, Finland",1001,POINT (24.93017 60.16837)
1,"Kaivokatu 8, 00101 Helsinki, Finland",1002,POINT (24.94189 60.16987)
2,"Hermanstads strandsväg 1, 00580 Helsingfors, F...",1003,POINT (24.97740 60.18736)
3,"Itäväylä, 00900 Helsinki, Finland",1004,POINT (25.09196 60.21448)
4,"Tyynenmerenkatu 9, 00220 Helsinki, Finland",1005,POINT (24.92148 60.15658)
